In [27]:
## sf_prefix="/Users/simon/research/projects/papers/xTEA_paper/NC_revision/round1_revision/genotype/"
sf_xTea_gntp=sf_prefix+"xTea_coordinate_ovlp.txt.true_positive._with_info2"
sf_PNAS_sites=sf_prefix+"PNAS_coordinate_ovlp.txt.true_positive._with_info"##
sf_PNAS_gntp=sf_prefix+"CEU_genotype_PNAS_Burns_Alu.txt"
i_slack=50

#first load in the xTea results
m_xTea_sites_gnpt={}
m_sample={}
l_sample=[]
with open(sf_xTea_gntp) as fin_rslts:
    for line in fin_rslts:
        fields=line.split()
        if line[:2]=="##":
            continue
        if fields[0]=="#CHROM":#
            for s_tmp in fields[9:]:
                m_sample[s_tmp]=1
                l_sample.append(s_tmp)
        else:
            s_chrm=fields[0]
            pos=int(fields[1])
            l_tmp_gntp=[]
            for s_gntp in fields[9:]:
                l_tmp_gntp.append(s_gntp)
            if s_chrm not in m_xTea_sites_gnpt:
                m_xTea_sites_gnpt[s_chrm]={}
            m_xTea_sites_gnpt[s_chrm][pos]=l_tmp_gntp

            
m_pnas_sites_cvt={}
with open(sf_PNAS_sites) as fin_pnas_sites:
    for line in fin_pnas_sites:
        #each line in format: chr1    56537669        56537670        Alu_019
        fields=line.split()
        s_chrm=fields[0]
        pos=int(fields[1])
        s_id=fields[-1]
        i_xTea_pos=-1
        #m_pnas_sites_cvt[s_id]=(s_chrm,pos)
        if s_chrm in m_xTea_sites_gnpt:
            for i_tmp_pos in range(pos-i_slack, pos+i_slack):
                if i_tmp_pos in m_xTea_sites_gnpt[s_chrm]:
                    i_xTea_pos=i_tmp_pos
                    break
        if i_xTea_pos>0:
            m_pnas_sites_cvt[s_id]=(s_chrm,i_xTea_pos)
#print(m_pnas_sites_cvt)         
####
#load in the sites gntp
m_pnas_sites_gntp={}
m_sample_index={}
n_sites=0
with open(sf_PNAS_gntp) as fin_pnas_gntp:
    for line in fin_pnas_gntp:
        fields=line.strip().split()
        i_cnt=1
        if fields[0].strip()=="ID":
            for s_sample in fields[1:]:
                m_sample_index[s_sample]=i_cnt
                i_cnt+=1
        else:
            s_id=fields[0]
            l_tmp_gntp=[]
            for s_tmp_sample in l_sample:#
                tmp_idx=m_sample_index[s_tmp_sample]
                #print(tmp_idx)
                if len(fields)<=tmp_idx:
                    print(fields, len(fields), tmp_idx)
                s_gntp_tmp=fields[tmp_idx]
                s_gntp="0/0"
                if s_gntp_tmp=="AT" or s_gntp_tmp=="TA":
                    s_gntp="0/1"
                elif s_gntp_tmp=="AA":
                    s_gntp="0/0"
                elif s_gntp_tmp=="TT":
                    s_gntp="1/1"
                else:
                    s_gntp="none"
                l_tmp_gntp.append(s_gntp)
            if s_id not in m_pnas_sites_cvt:
                #print(s_id+" cannot be found!")
                continue
            #print(s_id+" is found!")
            (s_chrm, i_xTea_pos)=m_pnas_sites_cvt[s_id]
            if s_chrm not in m_pnas_sites_gntp:
                m_pnas_sites_gntp[s_chrm]={}
            ##
            m_pnas_sites_gntp[s_chrm][i_xTea_pos]=l_tmp_gntp
            n_sites+=1
print(n_sites)
print(len(l_sample))
#print(len(m_pnas_sites_gntp)####
#
#compare results
import itertools
sf_out_merged="merged_results.txt"
with open(sf_out_merged,"w") as fout_merged:#
    fout_merged.write("chrm\tpos")
    for s_sample in l_sample:
        fout_merged.write("\t"+s_sample)
    fout_merged.write("\n")
####
#print(m_pnas_sites_gntp)
    n_total0=0
    n_total1=0
    n_total2=0
    n_no_info=0#
    n_ovlp_0=0
    n_ovlp_1=0
    n_ovlp_2=0
    n_0_1=0#
    n_0_2=0
    n_1_0=0
    n_1_2=0
    n_2_0=0
    n_2_1=0#
    for s_chrm in m_pnas_sites_gntp:
        for i_pos in m_pnas_sites_gntp[s_chrm]:
            if i_pos==196917116 or i_pos==135914485 or i_pos==34113146:
                continue
            n_wrong=0
            fout_merged.write(s_chrm+"\t"+str(i_pos))#
            for (s_gntp1,s_gntp2) in zip(m_pnas_sites_gntp[s_chrm][i_pos], m_xTea_sites_gnpt[s_chrm][i_pos]):
                fout_merged.write("\t"+s_gntp1+":"+s_gntp2)
                if s_gntp1=="none":
                    n_no_info+=1#
                    continue
                
                elif s_gntp1=="0/0":
                    n_total0+=1
                elif s_gntp1=="0/1":
                    n_total1+=1
                else:
                    n_total2+=1
                if s_gntp1==s_gntp2:
                    if s_gntp2=="0/0":
                        n_ovlp_0+=1
                    elif s_gntp2=="0/1" or s_gntp2=="1/0":
                        n_ovlp_1+=1
                    elif s_gntp2=="1/1":
                        n_ovlp_2+=1
                else:
                    n_wrong+=1
                    #print(s_gntp1)
                    if s_gntp1=="0/0" and s_gntp2=="0/1":
                        n_0_1+=1
                    elif s_gntp1=="0/0" and s_gntp2=="1/1":
                        n_0_2+=1
                    elif s_gntp1=="0/1" and s_gntp2=="0/0":
                        n_1_0+=1
                    elif s_gntp1=="0/1" and s_gntp2=="1/1":
                        n_1_2+=1
                    elif s_gntp1=="1/1" and s_gntp2=="0/0":
                        n_2_0+=1
                    elif s_gntp1=="1/1" and s_gntp2=="0/1":
                        n_2_1+=1
            print(s_chrm+"\t"+str(i_pos)+"\t"+str(n_wrong))
            fout_merged.write("\n")
    print(n_total0, n_total1, n_total2, n_no_info)
    print(n_ovlp_0,n_ovlp_1,n_ovlp_2)
    print(n_0_1, n_0_2, n_1_0, n_1_2, n_2_0, n_2_1)#

####chr6	135914485	42 #reference copy
####chr5	34113146	31 #like a reference copy
####

22
45
chr1	56537673	0
chr1	88743864	0
chr1	88753699	0
chr1	116548659	0
chr6	28546559	0
chr6	111828575	0
chr14	55329134	3
chr3	196072710	0
chr3	87471318	4
chr3	128676365	4
chr3	158372046	0
chr5	158376056	0
chr10	60475217	0
chr11	58599127	0
chr7	99156983	0
chr15	49317392	0
chr2	230252751	0
chr2	241422856	0
chr2	43852946	0
659 126 43 27
659 126 32
0 0 0 0 0 11
